# Libraries

In [1]:
import json
import os
import random

import numpy as np
import matplotlib.pyplot as plt
import cv2
import shapely as sh

# Functions

In [ ]:
def centroid_polygon(x: list, y: list) -> tuple[float, float, list]:
    """
    Calculate the centroid of a polygon defined by its vertices.

    :param x: x-coordinates of the polygon vertices
    :param y: y-coordinates of the polygon vertices

    :return: output[0] = x centroid, output[1] = y centroid and output[2] = coordinates of the polygon in tuple format
    """

    coords = []
    for i, j in zip(x, y):
        coords.append((i, j))
    polyg = sh.geometry.Polygon(coords)
    centre = polyg.centroid

    return centre.x, centre.y, coords


def transport_polygon(x: list, y: list, x_new: float, y_new: float) -> tuple[float, float]:
    """
    Translate a polygon defined by its vertices to a new position.

    :param x: x-coordinates of the polygon vertices
    :param y: y-coordinates of the polygon vertices
    :param x_new: new x-coordinate for the centroid of the polygon
    :param y_new: new y-coordinate for the centroid of the polygon

    :return: output[0] = new x-coordinates of the polygon vertices and output[1] = new y-coordinates of the polygon vertices
    """

    x_g, y_g, coords = centroid_polygon(x, y)
    dx = -x_g
    dy = -y_g

    # Translate the polygon to the origin
    transl_polygon_00 = sh.affinity.translate(sh.geometry.Polygon(coords), xoff=dx, yoff=dy)

    # Translate the polygon to the new position
    transl_polygon_xy = sh.affinity.translate(transl_polygon_00, xoff=x_new, yoff=y_new)

    # Extract the new x and y coordinates
    x_trans = [po[0] for po in transl_polygon_xy.exterior.coords]
    y_trans = [po[1] for po in transl_polygon_xy.exterior.coords]

    return x_trans, y_trans


def size_polygon(x: list, y: list) -> tuple[float, float]:
    """
    Calculate the size of a polygon defined by its vertices.

    :param x: x-coordinates of the polygon vertices
    :param y: y-coordinates of the polygon vertices

    :return: output[0] = length in x direction and output[1] = length in y direction
    """

    x_min = min(x)
    x_max = max(x)
    y_min = min(y)
    y_max = max(y)
    l_x = x_max - x_min
    l_y = y_max - y_min

    return l_x, l_y


def exclude_cont_bound(myconts: dict, n_x: float, n_y: float) -> dict:
    """
    Exclude contours that touch the boundary of the image.
    
    :param myconts: Dictionary of contours
    :param n_x: Width of the image or criterion for exclusion
    :param n_y: Height of the image or criterion for exclusion

    :return: Filtered dictionary of contours that do not touch the boundary
    """
    
    myconts_copy = myconts.copy()
    for i in myconts:
        i_contour = myconts[i]
        for j in range(len(i_contour)):
            x, y = i_contour[j][0]
            if x == 0 or y == 0 or x >= n_x or y >= n_y:
                save_or_not_save = False
                break
        else:
            save_or_not_save = True
        if save_or_not_save == False:
          del myconts_copy[i]
          
    return myconts_copy


def extract_contours(contours: dict) -> dict: 
    """
    Extract contours from a list of contours and return them as a dictionary.
    
    :param contours: Contours to be processed
    
    :return: Contour points. Each index of the dictionary corresponds to a contour.
    """

    myconts = {}
    for i, contour in enumerate(contours):
        myconts[str(i)] = np.array(contour)
        
    return myconts


def process_images_to_json(filepath: str, output_json: str, width: int = 2500, height: int = 2500) -> None:
    """
    Process images to extract contours and save them in a JSON file.

    :param filepath: Path to the image or directory containing images
    :param output_json: Path to the output JSON file
    :param width: Width of the images (default is 2500)
    :param height: Height of the images (default is 2500)
    """

    contours_json = {}
    image_area = width * height 

    if os.path.isdir(filepath):
        files = [os.path.join(filepath, f) for f in os.listdir(filepath) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    elif isinstance(filepath, (list, tuple)):
        files = filepath 
    else:
        files = [filepath] 

    # Read each image and process contours
    for filename in files:
        img = cv2.imread(filename)
        if img is None:
            print(f"Error to read image: {filename}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        # Get contour
        myconts = extract_contours(contours)

        # Exclude contours that touch the boundary
        myc = exclude_cont_bound(myconts, width-1, height-1)

        # Create a dictionary to save contours in JSON format
        image_key = os.path.basename(filename)
        contours_json[image_key] = {}

        # Image background size to aggregate-mortar
        l_xmax = []
        l_ymax = []
        for idx, contour in myc.items():
            coords = contour[:, 0, :]
            x_coords = coords[:, 0].tolist()
            y_coords = coords[:, 1].tolist()
            res = size_polygon(x_coords, y_coords)
            l_xmax.append(res[0])
            l_ymax.append(res[1])
        l_xback = max(l_xmax)
        l_yback = max(l_ymax)
        l_max = max([l_xback, l_yback])
        # Colocar o tamanho do background como o tamanho do maior contorno e img na base 2
        
        # Save contours and their areas in the JSON format
        for idx, contour in myc.items():
            coords = contour[:, 0, :]
            x_coords = coords[:, 0].tolist()
            y_coords = coords[:, 1].tolist()
            xg, yg, _ = centroid_polygon(x_coords, y_coords)
            # x_coords_trans, y_coords_trans = transport_polygon(x_coords, y_coords, l_max/2, l_max/2)
            area = cv2.contourArea(contour)
            q = float(area / image_area)
            contours_json[image_key][f"{idx}"] = {
                'x': x_coords,
                'y': y_coords,
                'q': q,
                'lmax': l_max,
                'xg': xg,
                'yg': yg
            }
    with open(output_json, 'w') as f:
        json.dump(contours_json, f, indent=4)
    print(f"Contours extracted and saved to {output_json}")


def plot_contours_from_json(json_file: str, keys_to_plot: list = None, width: int = 2500, height: int = 2500) -> None:
    """
    Plot contours from a JSON file in a figure with 3 subplots:
    - Left: all contours filled (random one in yellow)
    - Center: single random contour filled (in yellow)
    - Right: random contour centered in square patch

    :param json_file: Path to the JSON file containing contours
    :param keys_to_plot: List of keys (image names) to plot. If None, plot all keys
    :param width: Width of the images (default is 2500)
    :param height: Height of the images (default is 2500)
    """

    with open(json_file, 'r') as f:
        contours_data = json.load(f)

    keys = keys_to_plot if keys_to_plot else contours_data.keys()

    for image_name in keys:
        if image_name not in contours_data:
            print(f"Warning: {image_name} not found in contours data!")
            continue

        contours = contours_data[image_name]
        img_all = np.zeros((height, width, 3), dtype=np.uint8)  # RGB
        img_random = np.zeros((height, width, 3), dtype=np.uint8)

        contour_list = []
        q_values = []
        x_list = []
        y_list = []

        for contour in contours.values():
            x = np.array(contour["x"])
            y = np.array(contour["y"])
            pts = np.vstack((x, y)).T.astype(np.int32).reshape((-1, 1, 2))
            contour_list.append(pts)
            q_values.append(contour.get("q", 0))
            x_list.append(x.tolist())
            y_list.append(y.tolist())

        # Escolher contorno aleatório
        if not contour_list:
            print(f"Warning: No contours to display in {image_name}")
            continue

        random_idx = random.randint(0, len(contour_list) - 1)
        random_contour = contour_list[random_idx]
        q_random = q_values[random_idx]
        x_rand = x_list[random_idx]
        y_rand = y_list[random_idx]

        # Plot todos os contornos em cinza, o aleatório em amarelo
        for i, c in enumerate(contour_list):
            color = (255, 255, 0) if i == random_idx else (255, 255, 255)  # amarelo ou branco
            cv2.drawContours(img_all, [c], -1, color, thickness=cv2.FILLED)

        # Plot contorno aleatório em amarelo na imagem vazia
        cv2.drawContours(img_random, [random_contour], -1, (255, 255, 0), thickness=cv2.FILLED)

        # Criar patch centralizado
        canvas_size = 512
        patch_img = np.zeros((canvas_size, canvas_size, 3), dtype=np.uint8)
        x_trans, y_trans = transport_polygon(x_rand, y_rand, canvas_size // 2, canvas_size // 2)
        pts_centered = np.array(list(zip(x_trans, y_trans)), dtype=np.int32).reshape((-1, 1, 2))
        cv2.drawContours(patch_img, [pts_centered], -1, (255, 255, 0), thickness=cv2.FILLED)

        # Plot com 3 subplots
        fig, axes = plt.subplots(1, 3, figsize=(16, 6))
        axes[0].imshow(img_all)
        axes[0].set_title(f"All Contours - {image_name}")
        axes[0].axis('off')

        axes[1].imshow(img_random)
        axes[1].set_title(f"Random Contour - q = {q_random:.6f}")
        axes[1].axis('off')

        axes[2].imshow(patch_img)
        axes[2].set_title("Centralized Patch")
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()


def crop_contours(json_file: str, output_dir: str, canvas_size: int = 512) -> None:
    """
    Cria imagens com contornos centrados (de fato), transladando o contorno
    para o centro da imagem antes de desenhar.
    """

    os.makedirs(output_dir, exist_ok=True)

    with open(json_file, 'r') as f:
        contours_data = json.load(f)

    for image_name, contours in contours_data.items():
        for idx, contour in contours.items():
            x_coords = contour["x"]
            y_coords = contour["y"]
            l_max = contour["lmax"]

            # Transladar para centro da nova imagem
            x_new, y_new = canvas_size // 2, canvas_size // 2
            x_trans, y_trans = transport_polygon(x_coords, y_coords, x_new, y_new)

            # Criar imagem vazia
            blank = np.zeros((canvas_size, canvas_size), dtype=np.uint8)

            # Criar e desenhar contorno centralizado
            pts = np.array(list(zip(x_trans, y_trans)), dtype=np.int32).reshape((-1, 1, 2))
            cv2.drawContours(blank, [pts], -1, color=255, thickness=cv2.FILLED)

            # Crop final (opcional, mas aqui a imagem já está centrada)
            cropped = blank

            output_name = f"{os.path.splitext(image_name)[0]}_{idx}.png"
            cv2.imwrite(os.path.join(output_dir, output_name), cropped)

    print(f"Contornos centralizados e salvos em '{output_dir}'")


def flatten_contours_json(original_json_path: str, output_json_path: str) -> None:
    """
    Converte um JSON com estrutura por imagem em um dicionário plano com chave única para cada contorno.
    """

    with open(original_json_path, 'r') as f:
        data = json.load(f)

    flat_data = {}

    for image_name, contours in data.items():
        base_name = os.path.splitext(image_name)[0]  # remove .png
        for idx, contour in contours.items():
            key = f"{base_name}_{idx}.png"
            # Apenas x, y e q
            flat_data[key] = {
                "x": contour.get("x", []),
                "y": contour.get("y", []),
                "q": contour.get("q", None)
            }

    with open(output_json_path, 'w') as f:
        json.dump(flat_data, f, indent=4)

    print(f"Novo JSON salvo em '{output_json_path}' com {len(flat_data)} contornos.")

# Image or images

In [3]:
# filename = 'dataset/binary-masks/01.png' # Commen if you want to use a set of images
filepath = 'imgs_patchs_improved'

# Get contour

In [5]:
process_images_to_json(filepath, output_json="new_dataset_contours_aggregate.json")

# Verify the output
with open("new_dataset_contours_aggregate.json", 'r') as f:
    contours_data = json.load(f)
    print(f"Extracted contours from {len(contours_data)} images.")
    for image, contours in contours_data.items():
        print(f"{image}: {len(contours)} contours found. With the first proportion of area: {contours['0']['q']:.4f}")

Contours extracted and saved to new_dataset_contours_aggregate.json
Extracted contours from 1000 images.
patch512_00.png: 1 contours found. With the first proportion of area: 0.0003
patch512_01.png: 1 contours found. With the first proportion of area: 0.0003
patch512_02.png: 1 contours found. With the first proportion of area: 0.0001
patch512_03.png: 1 contours found. With the first proportion of area: 0.0001
patch512_04.png: 1 contours found. With the first proportion of area: 0.0001
patch512_05.png: 1 contours found. With the first proportion of area: 0.0001
patch512_06.png: 1 contours found. With the first proportion of area: 0.0001
patch512_07.png: 1 contours found. With the first proportion of area: 0.0002
patch512_08.png: 1 contours found. With the first proportion of area: 0.0002
patch512_09.png: 1 contours found. With the first proportion of area: 0.0001
patch512_10.png: 1 contours found. With the first proportion of area: 0.0002
patch512_100.png: 1 contours found. With the fir

In [ ]:
# crop_contours(json_file="dataset_contours_aggregate.json", output_dir="dataset/image_patchs")

Contornos centralizados e salvos em 'dataset/image_patchs'


In [ ]:
# flatten_contours_json(original_json_path="dataset_contours_aggregate.json", output_json_path="new_dataset_contours_aggregate.json")

Novo JSON salvo em 'new_dataset_contours_aggregate.json' com 7039 contornos.


# Plot the dataset

In [ ]:
# Create a function to select one contour in dataset and plot it
# Put this function on the cell "# Functions" (see above)

# plot_contours_from_json("dataset_contours_aggregate.json", keys_to_plot=["01.png", "04.png"])

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_contours_aggregate.json'